---
title: NBIS Support \#6668
subtitle: Shotgun metagenomics of gut microbiota in mice
author: John Sundh
date: last-modified
format:
    nbis-html: default
toc: true
toc-depth: 5
toc-expand: 3
jupyter: python3
---

**Description**

This project aims to study the effects of dibutyl phthalate (a plastic-derived contaminant) on the fecal microbiome composition and functional profile in three generations of mice (F0 "exposed mice" and F1 and F2 "offsprings"). Shotgun metagenomic sequencing of the DNA samples was done at NGI, Stockholm.

Specific goals include:

- To identify the taxonomic composition and profiling (species level), assess community diversity and characterize the relative abundances of taxa between the phthalates-treated mice vs. untreated mice (controls)
- Deep functional characterization of the phthalate-treated mice and control mice focusing on antibiotic resistance genes, virulence factors, carbohydrate metabolism, functional redundancy, etc
- To perform correlation analysis with other findings observed in F0, F1 and F2 mice, including immune, metabolic and liver phenotypes


In [2]:
import pandas as pd
import re
import seaborn as sns
import matplotlib_inline
import matplotlib.pyplot as plt
import h5py
import numpy as np
import umap
import altair as alt
alt.renderers.enable("html")
#from vega_datasets import data
from skbio.stats.composition import clr as skbio_clr
from sklearn.decomposition import PCA
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
sns.set_style("whitegrid")

In [3]:
def read_h5(filename):
    """
    Reads a hdf file with h5py package and returns a data frame
    """
    with h5py.File(filename, 'r') as hdf_file:
        data_matrix = hdf_file['data'][:]
        sample_names = hdf_file['data'].attrs['sample_names'].astype(str)
    return pd.DataFrame(dict(zip(sample_names, data_matrix)))

def clr(data, log=np.log2, method="custom"):
    """
    Convert counts data to centered log ratio with log2. 
    Zeros are replaced by multiplicative_replacement from scikit-bio. 
    See wikipedia for centered log ratio.
    Taken from https://github.com/metagenome-atlas/Tutorial/blob/master/Python/utils/mag_scripts.py
    
    Rows should be samples and columns should be features
    """

    from skbio.stats import composition

    # remove columns with all zeros
    data = data.loc[:, ~(data == 0).all()]

    # dataframe with replaced zeros
    data = pd.DataFrame(
        composition.multiplicative_replacement(data),
        columns=data.columns,
        index=data.index,
    )
    if method == "custom":
        data = log(data)
        return (data.T - data.mean(1)).T
    elif method == "skbio":
        _ = pd.DataFrame(skbio_clr(data))
        _.columns = data.columns
        _.index = data.index
        return _

## Project overview

As of 11 May 2023, roughly 20 hours (out of 80 hours contracted) had been used.

### Git repository

A GitHub repository was created for the project and is available at [https://github.com/NBISweden/SMS-23-6668-micegut](https://github.com/NBISweden/SMS-23-6668-micegut). This repository is used to track configuration files, sample lists, utility scripts and documentation for the project. Actual results will be shared elsewhere.

### Uppmax info

#### Analysis directory

The project GitHub repository listed above was cloned into `/proj/snic2020-5-486/nobackup/SMS-23-6668-micegut` on the Rackham cluster. Analysis was executed from this directory.

#### Raw data

Raw data is stored on Uppmax at:

`/proj/snic2020-5-486/dbp_gut_microbiome/DataDelivery_2023-01-10_15-17-23_ngisthlm00104/files/P27457/`

#### Compute account

SNIC 2022/5-350 / snic2022-5-350

## Analysis

### Sample file

A total of 108 samples were divided into 10 assembly groups:

In [4]:
sample_df = pd.read_csv("../data/sample_list_atlas.tsv", header=0, sep="\t", index_col=0)
sample_df.sort_values("BinGroup", inplace=True)
group_df = pd.DataFrame(sample_df.groupby("BinGroup").size(), columns=["samples"]).sort_index()
sample_dict = sample_df.to_dict(orient="index")
group_df.sort_index()

,samples
BinGroup,
F0_C,12
F0_H,12
F0_L,10
F1_C,8
F1_H,8
F1_L,6
F2_C,16
F2_H,16
F2_L,17


In [5]:
sample_info = sample_df.drop(["Reads_raw_R1","Reads_raw_R2"], axis=1)
sample_info.rename(columns = {'BinGroup': 'Group'}, inplace=True)
sample_info = sample_info.assign(Treatment=pd.Series([x.split("_")[-1] for x in sample_info.Group], index=sample_info.index))
sample_info = sample_info.assign(Generation=pd.Series([x[0:2] if x!="mock" else x for x in sample_info.Group], index=sample_info.index))
sample_info.index.name = "sample_id"
sample_info = sample_info.assign(Sample=pd.Series([x.split("-")[0] for x in sample_info.index], index=sample_info.index))

### Initial strategy

Initially I looked into using the [nf-core/mag](https://nf-co.re/mag) Nextflow pipeline for this project. However, after some trial runs I found that the pipline is likely not suited for this analysis. This could also be due to my lack of detailed knowledge and experience with debugging Nextflow pipelines but nevertheless I turned to other alternatives. My attempts at using `nf-core/mag` is documented at [doc/nf-core-mag.ipynb](https://github.com/NBISweden/SMS-23-6668-micegut/blob/main/doc/nf-core-mag.ipynb).

### ATLAS

As an alternative I tried [ATLAS](https://github.com/metagenome-atlas/atlas/), a workflow written in Snakemake that performs QC, Assembly, Binning and functional annotation of contigs using gene catalogs (genes are clustered with mmseqs and annotated with eggNOG).

#### Set up

To install I ran:

```bash 
export CONDARC="/proj/snic2020-5-486/nobackup/SMS-23-6668-micegut/.condarc"
mamba env create -f atlas-env.yml -p envs/atlas
```

As a starting point I copied the `template_config.yaml` file from the atlas package into `conf/atlas-config.yml`

```bash
cp $CONDA_PREFIX/lib/python3.10/site-packages/atlas/workflow/config/template_config.yaml conf/atlas-config.yml
```

A sample list was created with:

```bash
python src/make_sample_list.py atlas > data/sample_list_atlas.tsv
```

Set up output dir:

```bash
mkdir atlas
cd atlas
ln -s ../data/sample_list_atlas.tsv samples.tsv
```

#### Setting up cluster execution

To use the cluster profile for ATLAS I ran:
```bash
cookiecutter --output-dir ~/.config/snakemake https://github.com/metagenome-atlas/clusterprofile.git
```

and used defaults when prompted.

I then updated the `~/.config/snakemake/cluster/cluster_config.yaml` file to be:

```yaml
__default__:
  #queue: normal
  account: "snic2022-5-350"

rulename:
  queue: long
  account: ""
  time_min:  # min
  threads:
```

and set the `~/.config/snakemake/cluster/config.yaml` file to contain:

```yaml
restart-times: 0
cluster-config: "/domus/h1/john/.config/snakemake/cluster/cluster_config.yaml" #abs path
cluster: "scheduler.py" #
cluster-status: "slurm_status.py" #
max-jobs-per-second: 10
max-status-checks-per-second: 10
cores: 99 # how many jobs you want to submit to your cluster queue
local-cores: 1
rerun-incomplete: true  # recomended for cluster submissions
keep-going: true
reason: true
printshellcmds: true
```

#### Results

For the atlas workflow, `/proj/snic2020-5-486/nobackup/SMS-23-6668-micegut/atlas/` was used as an output directory.

##### QC

The `atlas run qc` command runs the BBTools suite to preprocess samples (incl. duplicate removal, quality trimming, adapter removal and phiX and host reads removal).

Here's how the qc command was executed:

```bash
atlas run -w /proj/snic2020-5-486/nobackup/SMS-23-6668-micegut/atlas -c /proj/snic2020-5-486/nobackup/SMS-23-6668-micegut/conf/atlas-config.yml --profile cluster qc
```

Because atlas doesn't come with fastqc/multiqc I added these dependencies to the atlas-env.yml file and updated the environment which installed `fastqc=0.12.1` and `multiqc=1.14`.

To run fastqc on all preprocessed R1/R2 fastq files:

```bash
mkdir -p atlas/fastqc
fastqc -o atlas/fastqc atlas/*/sequence_quality_control/*_R*fastq.gz
```

And then multiqc:

```bash
multiqc --outdir atlas/multiqc atlas/*/logs/QC/quality_filter.log atlas/fastqc/*.zip
```

I also added a small workflow file (`src/multiqc.smk`) to run these steps for all samples (using output from atlas):

```bash
snakemake -s src/multiqc.smk --config samples=atlas/samples.tsv --slurm --default-resources slurm_account=snic2022-5-350 -j 100
```

Checking the resulting `atlas/multiqc/multiqc_report.html` shows that all samples passed QC. Below are plots of total reads that passed QC in all samples.

In [6]:
mqc_df = pd.read_csv("../atlas/multiqc/multiqc_data/multiqc_general_stats.txt", header=0, sep="\t", index_col=0, usecols=[0,11])
mqc_df.rename(columns = lambda x: x.replace("FastQC_mqc-generalstats-fastqc-", ""), inplace=True)
mqc_df = mqc_df.loc[(mqc_df.index.str.contains("_R1"))&(~mqc_df.index.str.contains("deduplicated"))]
mqc_df.rename(index=lambda x: x.replace("_QC_R1", ""), inplace=True)
mqc_df = pd.merge(mqc_df, sample_info, left_index=True, right_index=True)
mqc_df.index.name="sample_id"

In [7]:
alt.Chart(mqc_df.reset_index(), title="Total sequences passing QC").mark_bar().encode(
    y="total_sequences",
    x=alt.X("sample_id").sort("color"),
    color="Group",
    tooltip=["sample_id","total_sequences","Group"]
).properties(
    width=600,
    height=200
)

alt.Chart(...)

In [8]:
alt.Chart(mqc_df, title="Sequences per group").mark_boxplot().encode(
    x="total_sequences",
    y=alt.Y("Group"),
    color="Treatment"
).properties(
    width=400,
    height=300
)

alt.Chart(...)

This shows that there are between ~12 - 30 M reads remaining after QC, with no obvious biases in terms of read numbers for samples/groups.

##### Assembly

The `atlas run assembly` command produces a fasta file for each sample, maps the QCd reads against the contigs to produce a bam file and also generates some stats.

Here's how atlas was executed:

```bash
atlas run -w /proj/snic2020-5-486/nobackup/SMS-23-6668-micegut/atlas -c /proj/snic2020-5-486/nobackup/SMS-23-6668-micegut/conf/atlas-config.yml --profile cluster assembly
```

The config file `conf/atlas-config.yml` was updated to give 20 threads and 128 GB memory to the megahit assembler. The assembly part of the pipeline finished in roughly 1.5 days.

**Assembly stats**

In [9]:
asm_stats = pd.read_csv("../atlas/stats/combined_contig_stats.tsv", header=0, sep="\t", index_col=0)
asm_stats = asm_stats.assign(AssemblySize=pd.Series([round(x/1000000, 2) for x in asm_stats.contig_bp], index=asm_stats.index))
asm_stats.index.name="sample_id"
asm_stats.rename(columns={'n_contigs': "Contigs", "AssemblySize": "Assembly size (Mbp)"}, inplace=True)
asm_stats = pd.merge(asm_stats, sample_info, left_index=True, right_index=True)

Common assembly statistics used here are:

* N50 = 50% of the assembly is contained in this many contigs (in descending length)
* L50 = length of the shortest contig in the set made up by `N50`
* Assembly size (Mbp) = Total assembly size in Mbp
* N_Predicted_Genes = number of genes predicted in the assembly by the `prodigal` gene caller
 
In general, a low N50 and a high L50 is preferrable.

Below is a plot of N50 vs. L50 values for all assemblies. Hover over the points for more information about the assembly. You can change the opacity and size of the points.

In [10]:
op_slider = alt.binding_range(min=0, max=1, step=0.05, name='opacity:')
op_var = alt.param(value=0.7, bind=op_slider)

si_slider = alt.binding_range(min=50, max=200, step=10, name="point size:")
si_var = alt.param(value=50, bind=si_slider)
si_var2 = alt.param(value=200, bind=si_slider)
dropdown_x = alt.binding_select(
    options=["N50","L50","Contigs","Assembly size (Mbp)", "N_Predicted_Genes"],
    name="X-axis column"
)
dropdown_y = alt.binding_select(
    options=["N50","L50","Contigs","Assembly size (Mbp)", "N_Predicted_Genes"],
    name="Y-axis column"
)
xcol_param = alt.param(
    value="L50",
    bind=dropdown_x
)
ycol_param = alt.param(
    value="N50",
    bind=dropdown_y
)

In [11]:
alt.Chart(asm_stats.reset_index()).mark_circle(size=si_var, opacity=op_var).encode(
    x="N50", y="L50", color="Treatment",
    tooltip=["sample_id","Contigs","Assembly size (Mbp)", "N50","L50", "N_Predicted_Genes", "Group"]
).add_params(
    op_var, si_var,
).properties(width=400, height=400)

alt.Chart(...)

Below is a chart where you can choose other statistics to plot against each other.

In [12]:
alt.Chart(asm_stats.reset_index()).mark_circle(size=si_var, opacity=op_var).encode(
    x=alt.X("x:Q").title('X-axis column'), y=alt.Y("y:Q").title("Y-axis column"), color="Treatment",
    tooltip=["sample_id","Contigs","Assembly size (Mbp)", "N50","L50", "N_Predicted_Genes", "Group"]
).transform_calculate(
    x=f'datum[{xcol_param.name}]',
    y=f'datum[{ycol_param.name}]'
).add_params(
    op_var, si_var, xcol_param, ycol_param
).properties(width=400, height=400)

alt.Chart(...)

The `mock` samples have high L50 values indicating assemblies with long contigs, but at the same time have a low number of predicted genes and low total assembly size. This is to be expected from samples with low complexity.

**Technical replicates**

Because we have technical replicates for a few samples we can look specifically at how these fair in the assembly and if the results are similar. Below are plots of assembly stats, but only showing samples with technical replicates.

In [13]:
tr_samples = asm_stats.loc[(asm_stats.index.str.contains("t.r"))|(asm_stats.index.str.startswith("m.c"))].Sample
trs = asm_stats.loc[asm_stats.Sample.isin(tr_samples)]

In [14]:
h = 300
w = 300
chart1 = alt.Chart(trs.reset_index()).mark_circle(size=si_var2, opacity=op_var).encode(
    x="L50", y="N50", color="Sample",
    tooltip=["sample_id","Contigs","Assembly size (Mbp)", "N50","L50", "N_Predicted_Genes", "Group"]
).add_params(
    op_var, si_var2,
).properties(width=w, height=h)

chart2 = alt.Chart(trs.reset_index()).mark_circle(size=si_var2, opacity=op_var).encode(
    x=alt.X("Contigs").title("#contigs"), y=alt.Y("contig_bp").title("Assembly size (bp)"), color="Sample",
    tooltip=["sample_id","Contigs","Assembly size (Mbp)", "N50","L50", "N_Predicted_Genes", "Group"]
).add_params(
    op_var, si_var2,
).properties(width=w, height=h)

#chart1 | chart2
alt.vconcat(chart1, chart2)

alt.VConcatChart(...)

In [15]:
alt.Chart(trs.reset_index()).mark_circle(size=si_var2, opacity=op_var).encode(
    x=alt.X("x:Q").title('X-axis column'), y=alt.Y("y:Q").title("Y-axis column"), color="Sample",
    tooltip=["sample_id","Contigs","Assembly size (Mbp)", "N50","L50", "N_Predicted_Genes", "Group"]
).transform_calculate(
    x=f'datum[{xcol_param.name}]',
    y=f'datum[{ycol_param.name}]'
).add_params(
    op_var, si_var2, xcol_param, ycol_param
).properties(width=400, height=400)

alt.Chart(...)

In summary,  the technical replicates have very similar assemblies, in terms of these statistics.

##### Annotation

The `atlas run genecatalog` step performs annotation of either assembled contigs or binned genomes. Because we haven't performed binning at this stage we will run the annotation using the assembled contigs as input.

Genes on contigs are first filtered to remove very short genes (<270 bp), followed by clustering of translated sequences from all samples using `mmseqs` at 90% identity threshold. Representatives sequences are then annotated using [eggNOG-mapper](http://eggnog-mapper.embl.de/).

Here is how atlas was executed:

```bash
atlas run -w /proj/snic2020-5-486/nobackup/SMS-23-6668-micegut/atlas -c /proj/snic2020-5-486/nobackup/SMS-23-6668-micegut/conf/atlas-config.yml --profile cluster genecatalog
```

**Results**

In total 2,744,382 gene clusters were generated in the `mmseqs` clustering step. Of these, 2,490,311 (90.7%) were given an annotation by eggNOG mapper.

In [39]:
#cov_stats = pd.read_csv("../atlas/Genecatalog/counts/sample_coverage_stats.tsv", sep="\t", index_col=0, header=0).T
#nmapped_reads = read_h5("../atlas/Genecatalog/counts/Nmapped_reads.h5")
median_coverage = read_h5("../atlas/Genecatalog/counts/median_coverage.h5")
gene_coverage_stats = pd.read_parquet("../atlas/Genecatalog/counts/gene_coverage_stats.parquet")
median_coverage.index = [x.split(" ")[0] for x in gene_coverage_stats["#Name"]]

eggNOG mapper assigns functions from a variety of databases using pre-computed orthologous groups. Below is a summary table of number of genes in the Genecatalog that were given a functional assignment from the different databases.

In [17]:
eggnog = pd.read_csv("../atlas/Genecatalog/annotations/eggNog.tsv.gz", header=0, sep="\t", index_col=0, comment="#")
features = ["COG_cat","GO_terms","EC","KO","KEGG_Pathway","KEGG_Module","KEGG_Reaction","KEGG_rclass","BRITE","KEGG_TC","CAZy","BiGG_Reaction","PFAMs"]
num_feat_dict = {}
for f in features:
    num_feat_dict[f] = {'Annotated genes': eggnog.loc[eggnog[f]!="-"].shape[0],
                        '% of Genecatalog': round((eggnog.loc[eggnog[f]!="-"].shape[0] / median_coverage.shape[0]) * 100, 1)}
num_feat_df = pd.DataFrame(num_feat_dict).T
num_feat_df.index.name="Feature"
num_feat_df.sort_values("Annotated genes", ascending=False)

,Annotated genes,% of Genecatalog
Feature,,
COG_cat,2296232.0,83.7
PFAMs,2262632.0,82.4
KO,1185647.0,43.2
BRITE,1185634.0,43.2
KEGG_Pathway,683256.0,24.9
EC,627862.0,22.9
KEGG_Reaction,453309.0,16.5
KEGG_rclass,437610.0,15.9
KEGG_Module,431877.0,15.7


In the `atlas/Genecatalog/counts` folder on Uppmax are sample-wise counts files for gene clusters and overall statistics files:

| file | contents | open with |
| ---- | -------- | ------- |
| gene_coverage_stats.parquet | name, length and GC of gene clusters | `pandas.read_parquet` |
| Nmapped_reads.h5 | reads mapped to each gene cluster in each sample | `h5py` |
| median_coverage.h5 | median fold coverage for each gene cluster in each sample | `h5py` |
| sample_coverage_stats.tsv | sum of coverage and total counts for all and non-zero genes for each sample | `pandas.read_csv` |

In addition, the `atlas/Genecatalog/alignments` directory contains individual coverage files for samples in the [parquet](https://www.databricks.com/glossary/what-is-parquet) format. These files look like this:

| GeneName | Avg_fold | Covered_percent | Read_GC | Median_fold | Std_Dev | Reads |
| -------- | -------- | --------------- | ------- | ----------- | ------- | ----- |
| Gene0000001 | 1.7038 | 69.8514 | 0.4857 | 1 | 1.66 | 13 |

The `Median_fold` values are the same as those reported for the sample in the `median_coverage.h5` file in the table above, while `Reads` corresponds to the values in the `Nmapped_reads.h5` file. The `.h5` files do not contain row names but are sorted in the same way as the `gene_coverage_stats.parquet` file so the names can be taken from there.

The atlas developer suggests to use the median coverage values to calculate relative abundance of each gene in each sample, followed by normalization using central log ratios. 

**MODIFICATION OF THE ATLAS SOURCE CODE**

Because atlas version 2.15.0 does not include any filtering of mapped reads I modified the atlas source code to allow passing `minmapq` to the `pileup.sh` script used when calculating coverages. These changes are tracked at [https://github.com/johnne/atlas](https://github.com/johnne/atlas). After these modifications I set `minimum_map_quality: 10` in the atlas config and removed some files in order to trigger a rerun with updated parameters:

```bash
# remove coverage stats from binning
rm -f atlas/*/binning/coverage/*_coverage_stats.txt
# remove Cobinning output
rm -rf atlas/Cobinning
# remove other binning output
rm -rf atlas/*/binning
# remove coverage files from Genecatalog
rm -f atlas/Genecatalog/alignments/*
# remove counts files from Genecatalog
rm -f atlas/Genecatalog/counts/*
# remove flag file for Genecatalog
rm -f atlas/finished_genecatalog
```

In [17]:
def umap_embedding(data, info_df, random_state=42, n_components=3, n_neighbors=15):
    """
    Perform UMAP fitting of data and return a dataframe with dimension coordinates
    as well as sample information
    """
    reducer = umap.UMAP(random_state=random_state, n_components=n_components, n_neighbors=n_neighbors)
    embedding = reducer.fit_transform(data)
    umap_df = pd.DataFrame(embedding, columns=[f"UMAP{x}" for x in range(1,n_components+1)], index=data.index)
    umap_df = pd.merge(umap_df, info_df, left_index=True, right_index=True)
    umap_df.index.name = "sample_id"
    return umap_df

def extract_annot(annot_df, cov_df, feature):
    """
    Extract coverage values for genes given certain annotations
    """
    feature_genes = list(annot_df.loc[annot_df[feature]!="-"].index)
    return cov_df.loc[feature_genes]    

def pca_fit(data, info_df, random_state=42, n_components=3):
    """
    Perform PCA fitting of data and return a dataframe with dimension coordinates
    as well as sample information
    """
    pca = PCA(n_components=n_components, random_state=random_state)
    transformed_data = pca.fit_transform(data)
    pca_df = pd.DataFrame(transformed_data, columns=[f"PC {x}" for x in range(1, n_components+1)], index=data.index)
    pca_df = pd.merge(pca_df, info_df, left_index=True, right_index=True)
    pca_df.index.name = "sample_id"
    return pca_df, pca.explained_variance_ratio_

Below we take a quick look at the annotation patterns in the samples. We will use both the full Genecatalog, as well as subsets of genes given an annotation in the KEGG, CAZy and PFAM databases. The median coverage is converted to relative abundance and normalized using central log ratio. Normalized coverage is then used as input to project samples into a 2-dimensional space, using [umap](https://umap-learn.readthedocs.io/en/latest/index.html) and PCA (implemented in [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)). Mock samples are excluded in this analysis.

In [18]:
# Normalize the median coverage

# Full Genecatalog
relab = median_coverage.div(median_coverage.sum())
relab_clr = clr(relab.T)

# KO subset
median_coverage_ko = extract_annot(eggnog, median_coverage, "KO")
relab_ko = median_coverage_ko.div(median_coverage_ko.sum())
relab_ko_clr = clr(relab_ko.T)

# CAZy subset
median_coverage_cazy = extract_annot(eggnog, median_coverage, "CAZy")
relab_cazy = median_coverage_cazy.div(median_coverage_cazy.sum())
relab_cazy_clr = clr(relab_cazy.T)

# PFAM subset
median_coverage_pfam = extract_annot(eggnog, median_coverage, "PFAMs")
relab_pfam = median_coverage_pfam.div(median_coverage_pfam.sum())
relab_pfam_clr = clr(relab_pfam.T)

In [19]:
# Run umap fitting

# Full Genecatalog
umap_df = umap_embedding(data=relab_clr.loc[~relab_clr.index.str.contains("m.c")], n_neighbors=15, 
                         info_df=sample_info)

# KO subset
umap_ko_df = umap_embedding(data=relab_ko_clr.loc[~relab_ko_clr.index.str.contains("m.c")],
                         info_df=sample_info)

# CAZy subset
umap_cazy_df = umap_embedding(data=relab_cazy_clr.loc[~relab_cazy_clr.index.str.contains("m.c")],
                         info_df=sample_info)

# PFAM subset
umap_pfam_df = umap_embedding(data=relab_pfam_clr.loc[~relab_pfam_clr.index.str.contains("m.c")],
                         info_df=sample_info)

In [20]:
# Concat all projections
umap_df["Database"] = ["Full"]*umap_df.shape[0]
umap_ko_df["Database"] = ["KO"]*umap_ko_df.shape[0]
umap_cazy_df["Database"] = ["CAZy"]*umap_cazy_df.shape[0]
umap_pfam_df["Database"] = ["PFAMs"]*umap_pfam_df.shape[0]

umap_concat = pd.concat([
    pd.melt(umap_df.reset_index(), id_vars=["Group", "Treatment","Generation","Sample","Database", "UMAP1","UMAP2","UMAP3"]),
    pd.melt(umap_ko_df.reset_index(), id_vars=["Group", "Treatment","Generation","Sample","Database", "UMAP1","UMAP2","UMAP3"]),
    pd.melt(umap_pfam_df.reset_index(), id_vars=["Group", "Treatment","Generation","Sample","Database", "UMAP1","UMAP2","UMAP3"]),
    pd.melt(umap_cazy_df.reset_index(), id_vars=["Group", "Treatment","Generation","Sample","Database", "UMAP1","UMAP2","UMAP3"])
])

**UMAP projections**

In [21]:
c1 = alt.Chart(umap_df.reset_index(), title="Genecatalog").mark_point().encode(
    x="UMAP1", y="UMAP2", color="Generation", shape="Treatment",
    tooltip = ["sample_id","Group","Treatment","Generation"]
)
c2 = alt.Chart(umap_pfam_df.reset_index(), title="PFAMs").mark_point().encode(
    x="UMAP1", y="UMAP2", color="Generation", shape="Treatment",
    tooltip = ["sample_id","Group","Treatment","Generation"]
)
c3 = alt.Chart(umap_ko_df.reset_index(), title="KEGG orthologs").mark_point().encode(
    x="UMAP1", y="UMAP2", color="Generation", shape="Treatment",
    tooltip = ["sample_id","Group","Treatment","Generation"]
)
c4 = alt.Chart(umap_cazy_df.reset_index(), title="CAZy").mark_point().encode(
    x="UMAP1", y="UMAP2", color="Generation", shape="Treatment",
    tooltip = ["sample_id","Group","Treatment","Generation"]
)

alt.vconcat((c1|c2), (c3|c4), data=umap_concat)

alt.VConcatChart(...)

For both the full Genecatalog and the annotated subsets there is a clear separation of samples according to mice generation. For the CAZy database the separation is less complete.

In [22]:
# Run PCA

# Full Genecatalog
pca_df, exp_var = pca_fit(data=relab_clr.loc[~relab_clr.index.str.contains("m.c")], info_df=sample_info)

# KO subset
pca_ko_df, exp_var_ko = pca_fit(data=relab_ko_clr.loc[~relab_ko_clr.index.str.contains("m.c")], info_df=sample_info)

# CAZy subset
pca_cazy_df, exp_var_cazy = pca_fit(data=relab_cazy_clr.loc[~relab_cazy_clr.index.str.contains("m.c")], info_df=sample_info)

# PFAM subset
pca_pfam_df, exp_var_pfam = pca_fit(data=relab_pfam_clr.loc[~relab_pfam_clr.index.str.contains("m.c")], info_df=sample_info)

In [23]:
# Concat all PCAs
pca_df["Database"] = ["Full"]*pca_df.shape[0]
pca_ko_df["Database"] = ["KO"]*pca_ko_df.shape[0]
pca_cazy_df["Database"] = ["CAZy"]*pca_cazy_df.shape[0]
pca_pfam_df["Database"] = ["PFAMs"]*pca_pfam_df.shape[0]

pca_concat = pd.concat([
    pd.melt(pca_df.reset_index(), id_vars=["Group", "Treatment","Generation","Sample","Database", "PC 1","PC 2","PC 3"]),
    pd.melt(pca_ko_df.reset_index(), id_vars=["Group", "Treatment","Generation","Sample","Database", "PC 1","PC 2","PC 3"]),
    pd.melt(pca_pfam_df.reset_index(), id_vars=["Group", "Treatment","Generation","Sample","Database", "PC 1","PC 2","PC 3"]),
    pd.melt(pca_cazy_df.reset_index(), id_vars=["Group", "Treatment","Generation","Sample","Database", "PC 1","PC 2","PC 3"])
])

**PCA projections**

In [24]:
c1 = alt.Chart(pca_df.reset_index(), title="Genecatalog").mark_point().encode(
    x=alt.X("PC 1").title(f"PC 1 ({round(exp_var[0]*100, 1)}%)"), 
    y=alt.Y("PC 2").title(f"PC 2 ({round(exp_var[1]*100, 1)}%)"),
    color="Generation", shape="Treatment",
    tooltip = ["sample_id","Group","Treatment","Generation"]
)
c2 = alt.Chart(pca_pfam_df.reset_index(), title="PFAMs").mark_point().encode(
    x=alt.X("PC 1").title(f"PC 1 ({round(exp_var_pfam[0]*100, 1)}%)"), 
    y=alt.Y("PC 2").title(f"PC 2 ({round(exp_var_pfam[1]*100, 1)}%)"),
    color="Generation", shape="Treatment",
    tooltip = ["sample_id","Group","Treatment","Generation"]
)
c3 = alt.Chart(pca_ko_df.reset_index(), title="KEGG orthologs").mark_point().encode(
    x=alt.X("PC 1").title(f"PC 1 ({round(exp_var_ko[0]*100, 1)}%)"), 
    y=alt.Y("PC 2").title(f"PC 2 ({round(exp_var_ko[1]*100, 1)}%)"),
    color="Generation", shape="Treatment",
    tooltip = ["sample_id","Group","Treatment","Generation"]
)
c4 = alt.Chart(pca_cazy_df.reset_index(), title="CAZy").mark_point().encode(
    x=alt.X("PC 1").title(f"PC 1 ({round(exp_var_cazy[0]*100, 1)}%)"), 
    y=alt.Y("PC 2").title(f"PC 2 ({round(exp_var_cazy[1]*100, 1)}%)"),
    color="Generation", shape="Treatment",
    tooltip = ["sample_id","Group","Treatment","Generation"]
)

alt.vconcat((c1|c2), (c3|c4), data=pca_concat)

alt.VConcatChart(...)

PCA projections give similar results to umap, but with less compact clustering of samples by generation. Note that the first two principal components explain <30% of the variance.

##### Binning (Work in progress)

Genome binning refers to the process of grouping assembled contigs into collections, or 'bins', that resemble the genomes in the original sample. This is done by comparing the composition (via frequencies of short nucleotide sequences, typically tetramers) of all contigs as well as their abundance distribution across samples. Because genomes are known to differ in their nucleotide composition (_e.g._ GC content) and because the abundance distribution of sequences (contigs) from a genome should follow the distribution patterns of the organism harboring the genome these two sources of information can be used as input to binning tools that perform unsupervised "binning" of contigs. The algorithms are never perfect though so additional checks are put in place to measure both the completeness of each bin (how much of the original genome was placed into the bin) as well contamination (how much of the bin comes from multiple sources of genomes). 

Some of the strengths of binning are that the collection of contigs in a bin can be more robustly annotated in terms of taxonomy (often using multiple single-copy core genes and phylogenetic placement into reference phylogenies), and also in terms of function because _e.g._ the metabolic potential of the bins can be estimated using multiple annotated enzymes, KEGG orthologs _etc._

The bins can also be quantified in the genome and a taxonomic composition can be created for each sample using the bin abundances.

The binning tools supported in Atlas are:

- Metabat2
- MaxBin2
- CONCOCT
- vamb

The `vamb` binner is a fairly new tool that uses deep learning to maximize the number of high quality bins. In the current config we use Metabat2, MaxBin2 and vamb. The output from these binners is then evaluated using [DASTool](https://github.com/cmks/DAS_Tool) to select a non-redundant set of hiqh-quality bins.

The `atlas run binning` step generates genome bins for each binner specified in the config file.

Atlas was executed like so (the `all` keyword species to run both the remaining binning and genome annotation).

```bash
atlas run -w /proj/snic2020-5-486/nobackup/SMS-23-6668-micegut/atlas -c /proj/snic2020-5-486/nobackup/SMS-23-6668-micegut/conf/atlas-config.yml --profile cluster all
```

### Taxonomic annotation

Taxonomy can be assigned directly for assembled contigs in various ways. For long 

## Next steps

### 2023-05-12
Atlas works well with this dataset and the annotations can easily be supplemented using additional databases and tools by running either on the assembled contigs, predicted genes, the clustered Genecatalog or on generated genome bins. Adding taxonomic annotations can be done at any of these, but is perhaps most suited to either the Genecatalog (via _e.g._ `mmseqs`) or genome bins (via `GTDB-TK`).

One thing to consider is that in the latest version of Atlas there is no filtering of mapped reads based on map quality or percent nucleotide identity. This means that all reads from a sample will show up as "mapped" to the assemblies regardless of how reliable the mapping is. A quick test showed that introducing a minimum map quality of 10 reduces mapped reads by ~1.5%. This will likely not affect the overall pattern but should remove very low-quality mappings.